## Popular Super Hero

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

<b>Create spark session</b>

In [16]:
spark = SparkSession.builder.appName("MostPopularSuperhero").getOrCreate()

<b>Creating schema</b>

In [17]:
schema = StructType([ \
                     StructField("id", IntegerType(), True), 
                     StructField("name", StringType(), True)])

<b>Load two different datasets</b>

In [18]:
names = spark.read.schema(schema).option("sep", " ").csv("file:///SparkCourse/Marvel-names.txt")

lines = spark.read.text("file:///SparkCourse/Marvel-graph.txt") #Using schema is not relevant

<b>Put the graph data in a dataframe</b>

In [19]:
connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))
    

<b>Extract most popular super hero from graph dataset</b>

In [20]:
mostPopular = connections.sort(func.col("connections").desc()).first()

<b>Match the hero ID with the names file and give result</b>

In [21]:
mostPopularName = names.filter(func.col("id") == mostPopular[0]).select("name").first()

print(mostPopularName[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-appearances.")


CAPTAIN AMERICA is the most popular superhero with 1933 co-appearances.
